In [11]:
import json
import psycopg2

In [16]:
host = "localhost"
dbname = "meli_ps"
user = "gabrielmedeiros"

connection = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user
)

In [18]:
cursor = connection.cursor()

In [19]:
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")

In [20]:
cursor.fetchall()

[('seller',),
 ('products',),
 ('address',),
 ('attributes',),
 ('shipping',),
 ('installments',)]

In [109]:
def insert_seller(cursor, connection, result):
    insert_query = """
        INSERT INTO seller (id, nickname) 
        VALUES (%s, %s)
        ON CONFLICT (id) DO NOTHING;
    """
    seller = result.get('seller')
    data = (seller.get('id'), seller.get('nickname'))
    cursor.execute(insert_query, data)
    connection.commit()
    
def insert_address(cursor, connection, result):
    insert_query = """
        INSERT INTO address (id, state_id, state_name, city_name) 
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (id) DO NOTHING;
    """
    address = result.get('address')
    data = (address.get('city_id'), address.get('state_id'), address.get('state_name'), address.get('city_name'))
    cursor.execute(insert_query, data)
    connection.commit()
    
def insert_shipping(cursor, connection, result, product_id):
    insert_query = """
        INSERT INTO shipping (product_id, store_pick_up, free_shipping, logistic_type, mode, benefits, promise, shipping_score) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (product_id) DO NOTHING;
    """
    shipping = result.get('shipping')
    data = (product_id, shipping.get('store_pick_up'), shipping.get('free_shipping'), shipping.get('logistic_type'),
           shipping.get('mode'), shipping.get('benefits'), shipping.get('promise'), shipping.get('shipping_score'))
    cursor.execute(insert_query, data)
    connection.commit()
    
def insert_product(cursor, connection, result):
    insert_query = """
        INSERT INTO products (
            id, seller_id, city_id, title, condition, thumbnail_id, catalog_product_id, 
            listing_type_id, sanitized_title, permalink, buying_mode, site_id, category_id, 
            domain_id, thumbnail, currency_id, order_backend, price, original_price, 
            sales_price_amount, sales_price_regular_amount, available_quantity, 
            official_store_id, use_thumbnail_id, accepts_mercadopago
        ) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
                %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (id) DO NOTHING;
    """
    
    seller_id = result.get('seller', {}).get('id') if result.get('seller') else None
    city_id = result.get('address', {}).get('city_id') if result.get('address') else None
    sale_price_amount = result.get('sale_price', {}).get('amount') if result.get('sale_price') else None
    sale_price_regular_amount = result.get('sale_price', {}).get('regular_amount') if result.get('sale_price') else None
    
    data = (
        result.get('id'), 
        seller_id, 
        city_id, 
        result.get('title'), 
        result.get('condition'),
        result.get('thumbnail_id'), 
        result.get('catalog_product_id'), 
        result.get('listing_type_id'),
        result.get('sanitized_title'), 
        result.get('permalink'), 
        result.get('buying_mode'), 
        result.get('site_id'),
        result.get('category_id'), 
        result.get('domain_id'), 
        result.get('thumbnail'), 
        result.get('currency_id'),
        result.get('order_backend'), 
        result.get('price'), 
        result.get('original_price'), 
        sale_price_amount,
        sale_price_regular_amount, 
        result.get('available_quantity'), 
        result.get('official_store_id'),
        result.get('use_thumbnail_id'), 
        result.get('accepts_mercadopago')
    )

    cursor.execute(insert_query, data)
    connection.commit()

def insert_installments(cursor, connection, result, product_id):
    insert_query = """
        INSERT INTO installments (
            product_id, quantity, amount, rate, currency_id
        ) 
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT (product_id) DO NOTHING;
    """
    
    installments = result.get('installments')
    
    data = (
        product_id,
        installments.get('quantity'),
        installments.get('amount'),
        installments.get('rate'),
        installments.get('currency_id')
    )
    
    cursor.execute(insert_query, data)
    connection.commit()


In [110]:
for offset in range(10):
    with open(f'response_data/response_data_{offset}.json', 'r') as file:
        data = json.load(file)
    print(f'Starting file {offset}')
    count = 1
    total = len(data['results'])
    for result in data['results']:
        print(f'Starting result {count}/{total}')
        product_id = result['id']
        try:
            insert_seller(cursor, connection, result)
            insert_address(cursor, connection, result)
            insert_product(cursor, connection, result)
            insert_shipping(cursor, connection, result, product_id)
            insert_installments(cursor, connection, result, product_id)
            print(f'Result {count} successfull')
        except Exception as e:
            connection.rollback()
            print(f'Error on result {count} of file {offset}')
            print(e)
        count += 1

Starting file 0
Starting result 1/49
Result 1 successfull
Starting result 2/49
Result 2 successfull
Starting result 3/49
Result 3 successfull
Starting result 4/49
Error on result 4 of file 0
'NoneType' object has no attribute 'get'
Starting result 5/49
Result 5 successfull
Starting result 6/49
Result 6 successfull
Starting result 7/49
Result 7 successfull
Starting result 8/49
Result 8 successfull
Starting result 9/49
Result 9 successfull
Starting result 10/49
Result 10 successfull
Starting result 11/49
Result 11 successfull
Starting result 12/49
Result 12 successfull
Starting result 13/49
Result 13 successfull
Starting result 14/49
Result 14 successfull
Starting result 15/49
Result 15 successfull
Starting result 16/49
Result 16 successfull
Starting result 17/49
Result 17 successfull
Starting result 18/49
Result 18 successfull
Starting result 19/49
Result 19 successfull
Starting result 20/49
Result 20 successfull
Starting result 21/49
Result 21 successfull
Starting result 22/49
Result 2

Result 38 successfull
Starting result 39/50
Result 39 successfull
Starting result 40/50
Result 40 successfull
Starting result 41/50
Result 41 successfull
Starting result 42/50
Result 42 successfull
Starting result 43/50
Result 43 successfull
Starting result 44/50
Result 44 successfull
Starting result 45/50
Result 45 successfull
Starting result 46/50
Result 46 successfull
Starting result 47/50
Result 47 successfull
Starting result 48/50
Result 48 successfull
Starting result 49/50
Result 49 successfull
Starting result 50/50
Result 50 successfull
Starting file 6
Starting result 1/50
Result 1 successfull
Starting result 2/50
Result 2 successfull
Starting result 3/50
Result 3 successfull
Starting result 4/50
Error on result 4 of file 6
'NoneType' object has no attribute 'get'
Starting result 5/50
Result 5 successfull
Starting result 6/50
Result 6 successfull
Starting result 7/50
Result 7 successfull
Starting result 8/50
Result 8 successfull
Starting result 9/50
Result 9 successfull
Starting

In [113]:
cursor.execute("SELECT * FROM address")

In [114]:
cursor.fetchall()

[('BR-SP-44', 'BR-SP', 'São Paulo', 'São Paulo'),
 ('TUxCQ0pVSWNlZjAw', 'BR-MG', 'Minas Gerais', 'Juiz de Fora'),
 ('TUxCQ0VYVDdjZTBi', 'BR-MG', 'Minas Gerais', 'Extrema'),
 ('BR-SP-42', 'BR-SP', 'São Paulo', 'Campinas'),
 ('BR-SP-95', 'BR-SP', 'São Paulo', 'Barueri'),
 ('TUxCQ0xPTmI2Njg1', 'BR-PR', 'Paraná', 'Londrina'),
 ('BR-SP-11', 'BR-SP', 'São Paulo', 'Itapevi'),
 ('TUxCQ1BBTGUwMTUw', 'BR-SC', 'Santa Catarina', 'Palhoça'),
 ('QlItUkpRdWVpbWFkb3M', 'BR-RJ', 'Rio de Janeiro', 'Queimados'),
 ('BR-SP-23', 'BR-SP', 'São Paulo', 'Ribeirão Preto'),
 ('QlItU1BQcmVzaWRlbnRlIEVwaXTDoWNpbw',
  'BR-SP',
  'São Paulo',
  'Presidente Epitácio'),
 ('BR-SP-14', 'BR-SP', 'São Paulo', 'Jundiaí'),
 ('TUxCQ0JSQTU2M2Yy', 'BR-DF', 'Distrito Federal', 'Brasília'),
 ('TUxCQ0ZFSWI0MDkz', 'BR-BA', 'Bahia', 'Feira de Santana'),
 ('QlItU1BDZXJxdWlsaG8', 'BR-SP', 'São Paulo', 'Cerquilho'),
 ('QlJGcmFuY28gZGEgUm9jaGE', 'BR-SP', 'São Paulo', 'Franco da Rocha'),
 ('TUxCQ0NBTTZkYzBl', 'BR-SC', 'Santa Catarina', 